In [1]:
# !pip install xmltodict
# !pip install pytesseract
# !pip install pillow
# !pip install --force-reinstall opencv-python

In [2]:
import pandas as pd
import numpy as np
import xmltodict as x2d
from pathlib import Path
import json
import pytesseract as ptr
import cv2 as cv
from PIL import Image

ModuleNotFoundError: No module named 'xmltodict'

### (PART 1) Converting bounding box formats. 

As part of your Data Pipeline class, using the sample license plate recognition dataset, write two functions that takes in a single file (.xml) and converts:

1) PASCAL VOC format to YOLO format (e.g., **convert_pascal_to_yolo_format(filename='<path/to/file>/testfile.xml'))**

2) PASCAL VOC format to COCO format (e.g., **convert_pascal_to_coco_format(filename='<path/to/file>/testfile.xml'))**

Then, using these functions above, write a script that sequentially parses all PASCAL VOC annotation format .xml files in a directory and produce:

1) YOLO formatted annotations

2) COCO formatted annotations

Only submit the COCO annotations.

In [ ]:
def parse2dict(imxmldir):
    """
    Read all the PASCAL VOC xml files and convert to python dicts.
    
    used by:
    VOC2YOLOCOCO()
    
    parameters:
    imxmldir (str): The path to the images and PASCAL VOC xml files directory
    
    returns:
    datdicts (list of dict): The PASCAL VOC xml files as python dicts.
    """
    imxmlpth = Path(imxmldir)
    datdicts = []
    for xmlpth in sorted(imxmlpth.rglob('*.xml')):
        with open(xmlpth, 'r', encoding='utf-8') as xmlfile:
            xmldat = xmlfile.read()
            datdict = x2d.parse(xmldat, encoding='utf-8')
            datdicts.append(datdict)
#     print(datdicts[0])
    
    return datdicts

In [ ]:
def convert_pascal_to_yolo_format(datdicts):
    """
    Write each PASCAL VOC xml file data as a YOLO txt file.
    
    used by:
    VOC2YOLOCOCO()
    
    parameters:
    datdicts (list of dict): The PASCAL VOC xml files as python dicts.
    """
    for datdict in datdicts:
        fname = datdict['annotation']['filename'].split('.')[0] + '.txt'
        xmax = int(datdict['annotation']['object']['bndbox']['xmax'])
        xmin = int(datdict['annotation']['object']['bndbox']['xmin'])
        ymax = int(datdict['annotation']['object']['bndbox']['ymax'])
        ymin = int(datdict['annotation']['object']['bndbox']['ymin'])
                     
        width = int(xmax - xmin)
        height = int(ymax - ymin)
        xcent = int((width / 2) + xmin)
        ycent = int((height / 2) + ymin)
        
        line = '0 {} {} {} {}'.format(xcent, ycent, width, height)
                     
        yolopth = Path('../COCO_YOLO_dat/{}'.format(fname))
        with open(yolopth, 'w') as yolofile:
            yolofile.write(line)
        

In [ ]:
def coco_extract(datdict, anid_count):
    """
    Extract the COCO format information from the PASCAL VOC data.
    
    used by:
    convert_pascal_to_coco_format()
    
    parameters:
    datdict (dict): A PASCAL VOC xml file as a python dict.
    anid_count (int): The number of annotations read thus far in the xml directory.
    
    returns:
    cocodat (list of int, str): The COCO format information
    """
    file_name = datdict['annotation']['filename']
    imid = file_name.split('.')[0]
    xmax = int(datdict['annotation']['object']['bndbox']['xmax'])
    xmin = int(datdict['annotation']['object']['bndbox']['xmin'])
    ymax = int(datdict['annotation']['object']['bndbox']['ymax'])
    ymin = int(datdict['annotation']['object']['bndbox']['ymin'])
    width = datdict['annotation']['size']['width']
    height = datdict['annotation']['size']['height']

    anid = anid_count
    anid_count += 1
    image_id = imid
    category_id = 0
    bbox = [xmin, ymin, xmax - xmin, ymax - ymin]

    name = 'license plate'
    supercategory = 'vehicle'

    cocodat = [imid, width, height, file_name, anid, image_id, category_id,
            bbox, name, supercategory, anid_count]

    return cocodat

In [ ]:
def convert_pascal_to_coco_format(datdicts, platetrudir):
    """
    Write all PASCAL VOC xml file data into a COCO json file.
    
    Also writes the manually annotated license plate numbers.
    
    used by:
    VOC2YOLOCOCO()
    
    parameters:
    datdicts (list of dict): The PASCAL VOC xml files as python dicts.
    """
    platetrupth = Path(platetrudir)
    # load the manually annotated license plate numbers
    with open(platetrupth, 'r', encoding='utf-8') as platetrufile:
        platetrudict = json.load(platetrufile)
    
    anid_count = 0
    cocodict = {'images': [],
               'annotations': [],
               'categories': []}
    # parse the COCO data into dictionary format
    for datdict in datdicts:
        cocodat = coco_extract(datdict, anid_count)
        images = {'id': cocodat[0],
                 'width': cocodat[1],
                 'height': cocodat[2],
                 'file_name': cocodat[3]}
        cocodict['images'].append(images)
        annotations = {'id': cocodat[4],
                      'image_id': cocodat[5],
                      'category_id': cocodat[6],
                      'bbox': cocodat[7],
                      'name': cocodat[8],
                      'supercategory': cocodat[9],
                      'content': platetrudict[cocodat[0]]}  # plate numbers
        cocodict['annotations'].append(annotations)
        anid_count = cocodat[10]
    
    # write the COCO json file
    cocopth = Path('../COCO_YOLO_dat/COCO_annot.json')
    with open(cocopth, 'w') as jsonfile:
        json.dump(cocodict, jsonfile)
        

In [ ]:
def VOC2YOLOCOCO(imxmldir, outputdir, platetrudir):
    """
    Write the PASCAL VOC xml files as both YOLO txt files and a COCO json file.
    
    uses:
    parse2dict(), convert_pascal_to_yolo_format(), convert_pascal_to_coco_format()
    
    parameters:
    imxmldir (str): The path to the images and PASCAL VOC xml files directory
    outputdir (str): The path to the YOLO and COCO output file directory
    """
    datdicts = parse2dict(imxmldir)
    convert_pascal_to_yolo_format(datdicts)
    convert_pascal_to_coco_format(datdicts, platetrudir)

VOC2YOLOCOCO('../images', '../COCO_YOLO_dat', '../COCO_YOLO_dat/annotxmls.json')

### (PART 2) Reading License Plate numbers. 

Using the generated YOLO and COCO bounding boxes, write a function that crops the license-plate-only portion of the image as indicated by the ground truth bounding boxes. Display the original image and the output. Then, write a function that uses Google's Tasseract OCR that takes in a cropped image and returns the license plate number as a string.

In [ ]:
def get_plates(cocopthstr):
    """
    Crop the license plate out of a vehicle image and get the plate number.
    
    parameters:
    cocopthstr (str): The path to the COCO json file
    """
    cocopth = Path(cocopthstr)
    with open(cocopth) as jsonfile:
        cocodat = json.load(jsonfile)

    for annotation in cocodat['annotations']:
        image = Image.open('../images/{}.jpeg'.format(annotation['image_id']))
        left = annotation['bbox'][0]
        up = annotation['bbox'][1]
        right = annotation['bbox'][0] + annotation['bbox'][2]
        bottom = annotation['bbox'][1] + annotation['bbox'][3]
        cropim = image.crop((left, up, right, bottom))
        plate = ptr.image_to_string(cropim).replace(' ', '')
        plate = ''.join(filter(str.isalnum, plate))

        image.show()
        cropim.show()
        print(annotation['image_id'])
        print(plate)
        print('\n\n')

**As requested, the following cell will display the original image, cropped image, and string output.  This will apply to all 200+ images.**

In [ ]:
# get_plates('../COCO_YOLO_dat/COCO_annot.json')